# Phase 1: Naive Model - End-to-End Pipeline

**Objective:** Build a complete ML pipeline, even if simple. Get something working first.

## What we'll do:
1. Create labels (target: is the trade profitable?)
2. Create simple features (price-based only)
3. Train a logistic regression with walk-forward validation
4. Evaluate: Does our filter improve the baseline?

## Key Concepts:
- **No lookahead bias**: Features use only past data, labels use future data
- **Walk-forward validation**: Train on past, test on future, slide window
- **Meta-strategy**: We're filtering baseline signals, not generating new ones

---

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# ML imports
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Add src to path
import sys
sys.path.insert(0, str(Path.cwd().parent / 'src'))

# Import our utilities
from data import load_all_data, compute_returns
from features import compute_rolling_returns, compute_rolling_volatility, create_feature_matrix
from labels import create_forward_return_labels, create_cost_adjusted_labels, analyze_label_distribution
from metrics import compute_all_metrics, compare_strategies
from backtesting import (compute_strategy_returns, compute_portfolio_returns, 
                         compute_equity_curve, apply_ml_filter)

## 1. Load and Prepare Data

In [ ]:
# Load data
trade_log, prices, glassnode = load_all_data()

# Align trade_log and prices
common_idx = trade_log.index.intersection(prices.index)
common_assets = trade_log.columns.intersection(prices.columns)

signals = trade_log.loc[common_idx, common_assets]
prices_aligned = prices.loc[common_idx, common_assets]

print(f"Aligned data: {len(signals)} timestamps, {len(common_assets)} assets")
print(f"Assets: {list(common_assets)}")
print(f"Period: {signals.index.min()} to {signals.index.max()}")

---

## 2. Create Labels

**The label answers:** "If we follow the baseline signal at time t, will it be profitable?"

For simplicity:
- Label = 1 if forward return > 0 (profitable)
- Label = 0 if forward return <= 0 (not profitable)

We only create labels when the baseline signal = 1 (long), because when signal = 0, we're in cash anyway.

In [ ]:
# Create labels with different horizons
HORIZON = 8  # 8 periods = 1 day (3h * 8 = 24h)
COST_THRESHOLD = 0.002  # 0.2% to cover round-trip costs

labels = create_cost_adjusted_labels(
    prices_aligned, 
    signals,
    horizon=HORIZON,
    entry_cost=0.001,
    exit_cost=0.001
)

print(f"Labels shape: {labels.shape}")
print(f"\nLabel distribution:")
label_stats = analyze_label_distribution(labels, signals)
print(f"  Overall positive rate: {label_stats['overall']['positive_rate']*100:.1f}%")
print(f"  Total samples: {label_stats['overall']['total']}")

In [ ]:
# Per-asset label distribution
print("\nPer-asset label distribution:")
print("-" * 50)
for asset, stats in label_stats['per_asset'].items():
    print(f"{asset:6s}: {stats['positive_rate']*100:5.1f}% positive ({stats['total']} samples)")

---

## 3. Create Features

**Simple features first** - we can add complexity later.

Features at time t (using only information available at t):
- Returns over last N periods
- Volatility over last N periods
- Current signal state

In [ ]:
# Create simple features
# Return windows in 3-hour periods: 1=3h, 8=1d, 56=1w, 224=1m
return_features = compute_rolling_returns(prices_aligned, windows=[1, 8, 56, 224])
volatility_features = compute_rolling_volatility(prices_aligned, windows=[56, 224])

# Combine features
features = pd.concat([return_features, volatility_features], axis=1)

print(f"Feature matrix shape: {features.shape}")
print(f"\nFeatures created:")
for col in features.columns[:10]:
    print(f"  - {col}")
if len(features.columns) > 10:
    print(f"  ... and {len(features.columns) - 10} more")

In [ ]:
# Check for missing values
missing_pct = features.isna().sum() / len(features) * 100
print("Missing values (top 10):")
print(missing_pct.sort_values(ascending=False).head(10))

---

## 4. Prepare Data for Modeling

We need to:
1. Stack data across assets (one model for all assets)
2. Align features and labels
3. Drop missing values

In [ ]:
def prepare_stacked_data(features, labels, signals):
    """
    Prepare data by stacking across assets.
    Returns X (features) and y (labels) with MultiIndex (timestamp, asset).
    
    IMPORTANT: Feature columns are renamed to be asset-agnostic
    (e.g., 'ADA_return_1p' becomes 'return_1p') so that all assets
    have the same feature names and can be stacked without NaN issues.
    """
    data_rows = []
    
    for timestamp in labels.index:
        if timestamp not in features.index:
            continue
            
        for asset in labels.columns:
            # Only include rows where signal = 1 (we're considering a long)
            if signals.loc[timestamp, asset] != 1:
                continue
                
            label_val = labels.loc[timestamp, asset]
            if pd.isna(label_val):
                continue
            
            # Get asset-specific features
            feature_cols = [c for c in features.columns if c.startswith(asset + '_')]
            if not feature_cols:
                continue
                
            row_features = features.loc[timestamp, feature_cols]
            
            if row_features.isna().any():
                continue
            
            # Rename features to be asset-agnostic (remove asset prefix)
            # e.g., 'ADA_return_1p' -> 'return_1p'
            renamed_features = {}
            for col in feature_cols:
                new_name = col.replace(asset + '_', '')
                renamed_features[new_name] = row_features[col]
            
            data_rows.append({
                'timestamp': timestamp,
                'asset': asset,
                'label': label_val,
                **renamed_features
            })
    
    df = pd.DataFrame(data_rows)
    df = df.set_index(['timestamp', 'asset'])
    
    y = df['label']
    X = df.drop('label', axis=1)
    
    return X, y

X, y = prepare_stacked_data(features, labels, signals)
print(f"Prepared data: X shape = {X.shape}, y shape = {y.shape}")
print(f"Feature columns: {list(X.columns)}")
print(f"Label distribution: {y.value_counts().to_dict()}")
print(f"Positive rate: {y.mean()*100:.1f}%")

---

## 5. Walk-Forward Validation

**Critical**: We must use temporal splits, not random splits.

Walk-forward scheme:
```
Time: ----1----2----3----4----5----6----7----8---->

Fold 1: [TRAIN TRAIN TRAIN][TEST]
Fold 2:      [TRAIN TRAIN TRAIN][TEST]
Fold 3:           [TRAIN TRAIN TRAIN][TEST]
```

In [ ]:
def walk_forward_cv(X, y, train_size, test_size, step_size=None):
    """
    Generate walk-forward cross-validation splits.
    
    Args:
        X: Feature DataFrame with timestamp in index
        y: Label Series with timestamp in index
        train_size: Number of unique timestamps for training
        test_size: Number of unique timestamps for testing
        step_size: How many timestamps to step forward (default: test_size)
    
    Yields:
        (train_idx, test_idx, fold_info) tuples
    """
    if step_size is None:
        step_size = test_size
    
    # Get unique timestamps
    timestamps = X.index.get_level_values('timestamp').unique().sort_values()
    n_timestamps = len(timestamps)
    
    fold = 0
    start = 0
    
    while start + train_size + test_size <= n_timestamps:
        train_end = start + train_size
        test_end = train_end + test_size
        
        train_timestamps = timestamps[start:train_end]
        test_timestamps = timestamps[train_end:test_end]
        
        train_idx = X.index.get_level_values('timestamp').isin(train_timestamps)
        test_idx = X.index.get_level_values('timestamp').isin(test_timestamps)
        
        fold_info = {
            'fold': fold,
            'train_start': train_timestamps[0],
            'train_end': train_timestamps[-1],
            'test_start': test_timestamps[0],
            'test_end': test_timestamps[-1],
            'train_size': train_idx.sum(),
            'test_size': test_idx.sum()
        }
        
        yield train_idx, test_idx, fold_info
        
        start += step_size
        fold += 1

In [ ]:
# Define CV parameters
# Trade log has ~1200 timestamps, let's use:
# - 60% for initial training (~720 timestamps)
# - 10% test windows (~120 timestamps each)

n_timestamps = X.index.get_level_values('timestamp').nunique()
print(f"Total unique timestamps: {n_timestamps}")

TRAIN_SIZE = int(n_timestamps * 0.6)  # 60% for training
TEST_SIZE = int(n_timestamps * 0.1)   # 10% test windows
STEP_SIZE = TEST_SIZE                  # Non-overlapping test sets

print(f"Train size: {TRAIN_SIZE} timestamps")
print(f"Test size: {TEST_SIZE} timestamps")
print(f"Step size: {STEP_SIZE} timestamps")

# Count folds
n_folds = 0
for _ in walk_forward_cv(X, y, TRAIN_SIZE, TEST_SIZE, STEP_SIZE):
    n_folds += 1
print(f"Number of folds: {n_folds}")

---

## 6. Train Logistic Regression

Simple model first. We'll try more complex models later.

In [ ]:
def train_logistic_regression(X_train, y_train):
    """
    Train a logistic regression model with scaling.
    """
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_train)
    
    model = LogisticRegression(
        C=1.0,
        class_weight='balanced',  # Handle class imbalance
        max_iter=1000,
        random_state=42
    )
    model.fit(X_scaled, y_train)
    
    return model, scaler

In [ ]:
# Run walk-forward validation
all_predictions = []
all_actuals = []
fold_results = []

for train_idx, test_idx, fold_info in walk_forward_cv(X, y, TRAIN_SIZE, TEST_SIZE, STEP_SIZE):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    # Train model
    model, scaler = train_logistic_regression(X_train, y_train)
    
    # Predict probabilities
    X_test_scaled = scaler.transform(X_test)
    y_prob = model.predict_proba(X_test_scaled)[:, 1]
    y_pred = (y_prob > 0.5).astype(int)
    
    # Store predictions with index
    pred_df = pd.DataFrame({
        'probability': y_prob,
        'prediction': y_pred,
        'actual': y_test.values
    }, index=y_test.index)
    all_predictions.append(pred_df)
    
    # Compute fold metrics
    fold_info['accuracy'] = accuracy_score(y_test, y_pred)
    fold_info['precision'] = precision_score(y_test, y_pred, zero_division=0)
    fold_info['recall'] = recall_score(y_test, y_pred, zero_division=0)
    try:
        fold_info['auc'] = roc_auc_score(y_test, y_prob)
    except:
        fold_info['auc'] = 0.5
    
    fold_results.append(fold_info)
    
    print(f"Fold {fold_info['fold']}: Acc={fold_info['accuracy']:.3f}, "
          f"Prec={fold_info['precision']:.3f}, Rec={fold_info['recall']:.3f}, "
          f"AUC={fold_info['auc']:.3f}")

# Combine all predictions
predictions_df = pd.concat(all_predictions)

In [ ]:
# Summary of CV results
results_df = pd.DataFrame(fold_results)
print("\n" + "=" * 60)
print("WALK-FORWARD VALIDATION RESULTS")
print("=" * 60)
print(f"\nMean Accuracy:  {results_df['accuracy'].mean():.3f} (+/- {results_df['accuracy'].std():.3f})")
print(f"Mean Precision: {results_df['precision'].mean():.3f} (+/- {results_df['precision'].std():.3f})")
print(f"Mean Recall:    {results_df['recall'].mean():.3f} (+/- {results_df['recall'].std():.3f})")
print(f"Mean AUC:       {results_df['auc'].mean():.3f} (+/- {results_df['auc'].std():.3f})")

---

## 7. Evaluate Meta-Strategy

Now the key question: **Does filtering the baseline signals improve performance?**

We apply the ML filter:
```
signal_ml[t,a] = signal_base[t,a] × 1[p̂(t,a) > τ]
```

In [ ]:
def create_filtered_signals(predictions_df, signals, threshold=0.5):
    """
    Create filtered signals based on ML predictions.
    """
    # Get timestamps where we have predictions
    pred_timestamps = predictions_df.index.get_level_values('timestamp').unique()
    
    # Start with baseline signals for those timestamps
    filtered = signals.loc[pred_timestamps].copy()
    
    # Apply ML filter
    for (timestamp, asset), row in predictions_df.iterrows():
        if row['probability'] <= threshold:
            # Override signal to 0 (stay in cash)
            filtered.loc[timestamp, asset] = 0
    
    return filtered

In [ ]:
# Try different thresholds
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]

# Get timestamps where we have predictions
pred_timestamps = predictions_df.index.get_level_values('timestamp').unique()

# Baseline signals and prices for comparison period
baseline_signals = signals.loc[pred_timestamps]
comparison_prices = prices_aligned.loc[pred_timestamps]

# Baseline performance
baseline_returns = compute_strategy_returns(baseline_signals, comparison_prices, transaction_cost=0.001)
baseline_portfolio = compute_portfolio_returns(baseline_returns)
baseline_metrics = compute_all_metrics(baseline_portfolio.dropna())

print("BASELINE (During Test Period):")
print(f"  Sharpe Ratio:  {baseline_metrics['sharpe_ratio']:.3f}")
print(f"  Total Return:  {baseline_metrics['total_return']*100:.2f}%")
print(f"  Max Drawdown:  {baseline_metrics['max_drawdown']*100:.2f}%")

print("\n" + "=" * 60)
print("ML-FILTERED STRATEGY (Different Thresholds):")
print("=" * 60)

threshold_results = []

for threshold in thresholds:
    filtered_signals = create_filtered_signals(predictions_df, signals, threshold)
    
    # Compute returns
    filtered_returns = compute_strategy_returns(filtered_signals, comparison_prices, transaction_cost=0.001)
    filtered_portfolio = compute_portfolio_returns(filtered_returns)
    filtered_metrics = compute_all_metrics(filtered_portfolio.dropna())
    
    # Count trades
    n_trades_baseline = baseline_signals.diff().abs().sum().sum() / 2
    n_trades_filtered = filtered_signals.diff().abs().sum().sum() / 2
    
    result = {
        'threshold': threshold,
        'sharpe': filtered_metrics['sharpe_ratio'],
        'total_return': filtered_metrics['total_return'],
        'max_drawdown': filtered_metrics['max_drawdown'],
        'n_trades': n_trades_filtered,
        'trade_reduction': (n_trades_baseline - n_trades_filtered) / n_trades_baseline * 100
    }
    threshold_results.append(result)
    
    print(f"\nThreshold τ = {threshold}:")
    print(f"  Sharpe Ratio:    {result['sharpe']:.3f} (baseline: {baseline_metrics['sharpe_ratio']:.3f})")
    print(f"  Total Return:    {result['total_return']*100:.2f}%")
    print(f"  Max Drawdown:    {result['max_drawdown']*100:.2f}%")
    print(f"  Trade Reduction: {result['trade_reduction']:.1f}%")

In [ ]:
# Plot threshold analysis
threshold_df = pd.DataFrame(threshold_results)

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Sharpe ratio vs threshold
axes[0, 0].plot(threshold_df['threshold'], threshold_df['sharpe'], 'bo-', markersize=8)
axes[0, 0].axhline(y=baseline_metrics['sharpe_ratio'], color='r', linestyle='--', label='Baseline')
axes[0, 0].set_xlabel('Threshold (τ)')
axes[0, 0].set_ylabel('Sharpe Ratio')
axes[0, 0].set_title('Sharpe Ratio vs Threshold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Total return vs threshold
axes[0, 1].plot(threshold_df['threshold'], threshold_df['total_return'] * 100, 'go-', markersize=8)
axes[0, 1].axhline(y=baseline_metrics['total_return'] * 100, color='r', linestyle='--', label='Baseline')
axes[0, 1].set_xlabel('Threshold (τ)')
axes[0, 1].set_ylabel('Total Return (%)')
axes[0, 1].set_title('Total Return vs Threshold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Max drawdown vs threshold
axes[1, 0].plot(threshold_df['threshold'], threshold_df['max_drawdown'] * 100, 'ro-', markersize=8)
axes[1, 0].axhline(y=baseline_metrics['max_drawdown'] * 100, color='b', linestyle='--', label='Baseline')
axes[1, 0].set_xlabel('Threshold (τ)')
axes[1, 0].set_ylabel('Max Drawdown (%)')
axes[1, 0].set_title('Max Drawdown vs Threshold (Lower is Better)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Trade reduction vs threshold
axes[1, 1].plot(threshold_df['threshold'], threshold_df['trade_reduction'], 'mo-', markersize=8)
axes[1, 1].set_xlabel('Threshold (τ)')
axes[1, 1].set_ylabel('Trade Reduction (%)')
axes[1, 1].set_title('Trade Reduction vs Threshold')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 7.1 Best Threshold Equity Curve

In [ ]:
# Select best threshold (highest Sharpe)
best_threshold = threshold_df.loc[threshold_df['sharpe'].idxmax(), 'threshold']
print(f"Best threshold: {best_threshold} (Sharpe: {threshold_df['sharpe'].max():.3f})")

# Create filtered signals with best threshold
best_filtered_signals = create_filtered_signals(predictions_df, signals, best_threshold)

# Compute returns
best_filtered_returns = compute_strategy_returns(best_filtered_signals, comparison_prices, transaction_cost=0.001)
best_filtered_portfolio = compute_portfolio_returns(best_filtered_returns)

# Plot equity curves
fig, ax = plt.subplots(figsize=(14, 7))

baseline_equity = compute_equity_curve(baseline_portfolio.dropna())
filtered_equity = compute_equity_curve(best_filtered_portfolio.dropna())

ax.plot(baseline_equity.index, baseline_equity.values, 'b-', linewidth=2, label='Baseline', alpha=0.7)
ax.plot(filtered_equity.index, filtered_equity.values, 'g-', linewidth=2, label=f'ML Filtered (τ={best_threshold})')
ax.axhline(y=1, color='gray', linestyle='--', alpha=0.5)

ax.set_title('Equity Curve Comparison: Baseline vs ML-Filtered Strategy', fontsize=14)
ax.set_ylabel('Portfolio Value')
ax.set_xlabel('Date')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 8. Feature Importance Analysis

In [ ]:
# Train final model on all data to get feature importance
model_final, scaler_final = train_logistic_regression(X, y)

# Get feature importance (absolute coefficients)
importance = pd.DataFrame({
    'feature': X.columns,
    'coefficient': model_final.coef_[0],
    'abs_coefficient': np.abs(model_final.coef_[0])
}).sort_values('abs_coefficient', ascending=False)

print("Top 15 Most Important Features:")
print("-" * 50)
for i, row in importance.head(15).iterrows():
    direction = '+' if row['coefficient'] > 0 else '-'
    print(f"{row['feature']:40s} {direction} ({row['abs_coefficient']:.4f})")

In [ ]:
# Plot feature importance
fig, ax = plt.subplots(figsize=(10, 8))

top_features = importance.head(15)
colors = ['green' if c > 0 else 'red' for c in top_features['coefficient']]

ax.barh(range(len(top_features)), top_features['coefficient'], color=colors, alpha=0.7)
ax.set_yticks(range(len(top_features)))
ax.set_yticklabels(top_features['feature'])
ax.set_xlabel('Coefficient')
ax.set_title('Top 15 Feature Importance (Logistic Regression Coefficients)')
ax.axvline(x=0, color='black', linewidth=0.5)
ax.invert_yaxis()
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

---

## 9. Summary & Next Steps

### What we accomplished:
1. Built a complete ML pipeline: features → labels → model → evaluation
2. Implemented proper walk-forward validation (no lookahead)
3. Evaluated the meta-strategy against baseline

### Key findings:
- [ ] Fill in after running the notebook

### Next steps (Phase 2+):
1. **Better features**: Add Glassnode on-chain metrics, technical indicators
2. **Better models**: Try XGBoost, Random Forest
3. **Regime detection**: Use HMM to identify market regimes
4. **Threshold optimization**: More sophisticated threshold selection

In [ ]:
# Save predictions for later analysis
output_dir = Path.cwd().parent / 'data' / 'processed'
output_dir.mkdir(parents=True, exist_ok=True)

predictions_df.to_csv(output_dir / 'naive_model_predictions.csv')
print(f"Saved predictions to {output_dir / 'naive_model_predictions.csv'}")